# Cross-resonance gate between 2 transmons: strong drive regime

In this notebook we study the cross-resonance effect between 2 transmons in the strong drive regime.

In [65]:
import numpy as np
import scipy.integrate
import time
import qutip as qtp
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pysqkit
from pysqkit import QubitSystem
from pysqkit.drives.pulse_shapes import gaussian_top
from pysqkit.util.metrics import average_process_fidelity, \
    average_gate_fidelity
from pysqkit.util.phys import temperature_to_thermalenergy
from pysqkit.util.quantum import generalized_rabi_frequency
import pysqkit.util.transformations as trf
from pysqkit.util.linalg import get_mat_elem
from pysqkit.solvers.solvkit import integrate
from pysqkit.util.hsbasis import weyl_by_index
from pysqkit.solvers import solvkit
import qutip
from typing import List, Dict, Callable
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
#import util_cr as util
import copy
import json
import cmath

from IPython.display import display, Latex

The gate is based on the following driven Hamiltonian of a fluxonium and a transmon capacitively coupled

$$H = H^{(0)}_{T, 1} + H^{(0)}_{T, 2} + V + H_{drive},$$

$$H^{(0)}_{T, k} =  \hbar \omega_{T, k} \hat{b}_k^{\dagger}\hat{b}_k + \hbar \frac{\delta_{T, k}}{2} \hat{b}_k^{\dagger} \hat{b}_k^{\dagger}\hat{b}_k \hat{b}_k,$$

$$V = J_C \cdot \hat{q}_{T1} \cdot \hat{q}_{T2},$$

$$\frac{H_{drive}}{h} = \left( \varepsilon_{T1} \hat{q}_{T1} + \varepsilon_{T2} \hat{q}_{T2} \right) \cos(2\pi f_d t).$$

The (approximate) charge operator for the transmon in terms of annihilation and creation operators reads

$$
\hat{q}_{Tk} = i \biggl( \frac{E_{J,k}}{32 |\delta_{T,k} |} \biggr)^{1/4} (b_k^{\dagger} - b_k) = i q_{ZPF} (b_k^{\dagger} - b_k),
$$
where 
$$
E_{J, k} = \hbar \biggl(\frac{\omega_{T, k} - \delta_{T, k}}{8 | \delta_{T, k} |} \biggr)^2.
$$

In the following we introduce relaxation due to dielectric loss and pure dephasing on each qubit as error channels. 

The idea behind the cross-resonance gate is to drive one of the qubit at the frequency of the other using the fact that due to the coupling we have a $Z X$ term in the Hamiltonian. In particular, we will explore the idea of driving the fluxonium at the transmon frequency. The opposite idea to drive the transmon at the fluxonium frequency does not seem that promising. However, one might evaluate using Raman transitions.

# Introducing the qubits and the coupled system

In [66]:
with open('flx_transm_params.txt') as param_file:
    parameters_set = json.load(param_file)

In [163]:
temperature = 0.020 # K
thermal_energy = temperature_to_thermalenergy(temperature) # kb T/h in GHz
d_comp = 4

p_set = "1"

#Transmon
levels_t = 3
transm_t = pysqkit.qubits.SimpleTransmon(
    label='T1', 
    max_freq=parameters_set[p_set]["max_freq_t"], 
    anharm=parameters_set[p_set]["anharm_t"],
    diel_loss_tan=parameters_set[p_set]["diel_loss_tan_t"], #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy,    
    dim_hilbert=levels_t,
    dephasing_times=parameters_set[p_set]["dephasing_times_t"]
)


transm_c = pysqkit.qubits.SimpleTransmon(
    label='T2', 
    max_freq=parameters_set[p_set]["max_freq_t"] - 0.3, 
    anharm=parameters_set[p_set]["anharm_t"],
    diel_loss_tan=parameters_set[p_set]["diel_loss_tan_t"], #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy,    
    dim_hilbert=levels_t,
    dephasing_times=parameters_set[p_set]["dephasing_times_t"]
)


# We also add a drive on the fluxonium
transm_c.add_drive(
    pysqkit.drives.microwave_drive,
    label='cr_drive_c',
    pulse=pysqkit.drives.pulses.cos_modulation,
    pulse_shape=pysqkit.drives.pulse_shapes.gaussian_top
)

d_leak = levels_t*levels_t - d_comp

jc = parameters_set[p_set]["jc"]
coupled_sys = transm_t.couple_to(transm_c, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)

states_label = coupled_sys.all_state_labels()
states_dict = coupled_sys.states_as_dict(as_qobj=True)

In [164]:
state_label = ["00", "01", "10", "11"]
comp_states = {}
for label in state_label:
    state_tmp = coupled_sys.state(label)[1]
    loc = np.argmax(np.abs(state_tmp))
    phase = cmath.phase(state_tmp[loc])
    state_tmp = np.exp(-1j*phase)*state_tmp
    comp_states[label] = state_tmp

The basis principle behind the gate is illustrated in the following figure

The figure shows the energy of the bare levels. The orange arrows indicate the non-zero matrix elements between the (bare) levels provided by the capacitive coupling $V$: the more shaded the arrow the weaker the coupling. 
We look at the following quantities : 

$$\zeta_{ZZ} = E_{00} + E_{11} - E_{01} - E_{10}$$
$$\mu_{YZ} = \lvert \langle 10 | q_{F} | 00 \rangle - \langle 11 | q_{F} |01 \rangle \rvert $$
$$\mu_{Y} = \lvert \langle 10 | q_{F} | 00 \rangle + \langle 11 | q_{F} |01 \rangle \rvert $$

The second term quantifies the gate speed. Notice that if the phase of the drive is $0$ one hase a $YZ$ term from the charge operator. 

For the given parameters we have

In [165]:
def zz(system: QubitSystem) -> float:
    xi_zz = system.state('00')[0] + system.state('11')[0] \
        - system.state('01')[0] - system.state('10')[0]
    return xi_zz

def mu_yz_control(comp_states, op) -> float:
    yz0 = get_mat_elem(op, comp_states['00'], comp_states['10'])
    yz1 = get_mat_elem(op, comp_states['01'], comp_states['11'] )
    return (np.imag(yz0 - yz1))/2

def mu_yi_control(comp_states, op) -> float:
    yz0 = get_mat_elem(op, comp_states['00'], comp_states['10'] )
    yz1 = get_mat_elem(op, comp_states['01'], comp_states['11'] )
    return (np.imag(yz0 + yz1))/2

In [166]:
display(Latex(r'$\xi_{{ZZ}}/h = {:.3f} \, \mathrm{{MHz}}$'.format(1e3*zz(coupled_sys))))
q_op = coupled_sys["T2"].charge_op()
display(Latex(r'$\mu_{{YZ}} = {:.4f} $'.format(mu_yz_control(comp_states, q_op))))
display(Latex(r'$\mu_{{IY}} = {:.4f} $'.format(mu_yi_control(comp_states, q_op))))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

However this is a sensible analysis only at low drive strengths. Taking only the first 3 levels of the transmon and the first 4 of the fluxonium and in an interaction picture with 
$$
H_0 = \omega_d |1 \rangle \langle 1 |_{T1} + 2 \omega_d |2 \rangle \langle 2 |_{T1} + \omega_d |1 \rangle \langle 1 |_{T2} + 2 \omega_d |2 \rangle \langle 2 |_{T2}
$$
the Hamiltonian in the Rotating-Wave approximation is

$$
H_I = (\omega_{T1} - \omega_d)|1 \rangle \langle 1 |_{T1} + (\delta_{T1} + 2(\omega_{T1} - \omega_d))  |2 \rangle \langle 2 |_{T1} + (\omega_{T2} - \omega_d)|1 \rangle \langle 1 |_{T2} + (\delta_{T2} + 2(\omega_{T2} - \omega_d))  |2 \rangle \langle 2 |_{T2}  + J_C q_{ZPF, 1}q_{ZPF, 2}\biggl[\biggl(|1 \rangle \langle 0 |_{T1} + \sqrt{2}|2 \rangle \langle 1 |_{T1}  \biggr) \biggl(|0\rangle \langle 1 |_{T2} + \sqrt{2}|1 \rangle \langle 2 |_{T2}  \biggr)  + \mathrm{h.c.} \biggr] + \frac{\varepsilon_d}{2}i q_{ZPF, 2} \biggl(|1 \rangle \langle 0 |_{T2} + \sqrt{2} |2 \rangle \langle 1 |_{T2} - \mathrm{h.c.}    \biggr)
$$

In what follows we introduce this Hamiltonian directly in Qutip

In [167]:
freq_drive = transm_t.max_freq
eps_drive = 0.02


In [168]:
# Transmon 1 Hamiltonian
lev_t = 3
id_t1 = qtp.qeye(lev_t)
id_t2 = qtp.qeye(lev_t)
proj_1_t1 = qtp.basis(lev_t, 1)*qtp.basis(lev_t, 1).dag()
proj_2_t1 = qtp.basis(lev_t, 2)*qtp.basis(lev_t, 2).dag()
h_t1_single = (transm_t.max_freq - freq_drive)*proj_1_t1 + (transm_t.anharm + 2*(transm_t.max_freq - freq_drive))*proj_2_t1
h_t1 = qtp.tensor(h_t1_single, id_t2)

# Transmon 2 Hamiltonian
lev_t = 3
id_t1 = qtp.qeye(lev_t)
id_t2 = qtp.qeye(lev_t)
proj_1_t2 = qtp.basis(lev_t, 1)*qtp.basis(lev_t, 1).dag()
proj_2_t2 = qtp.basis(lev_t, 2)*qtp.basis(lev_t, 2).dag()
h_t2_single = (transm_c.max_freq - freq_drive)*proj_1_t2 + (transm_c.anharm + 2*(transm_c.max_freq - freq_drive))*proj_2_t2
h_t2 = qtp.tensor(id_t1, h_t2_single)

# Coupling Hamiltonian
coup_op_t1 = transm_t.charge_zpf*(qtp.basis(lev_t, 1)*qtp.basis(lev_t, 0).dag() + np.sqrt(2)*qtp.basis(lev_t, 2)*qtp.basis(lev_t, 1).dag())
coup_op_t2 = transm_c.charge_zpf*(qtp.basis(lev_t, 1)*qtp.basis(lev_t, 0).dag() + np.sqrt(2)*qtp.basis(lev_t, 2)*qtp.basis(lev_t, 1).dag())
h_coup = jc*(qtp.tensor(coup_op_t1, coup_op_t2) + qtp.tensor(coup_op_t1, coup_op_t2).dag())
 
# Drive Hamiltonian
h_drive = transm_c.charge_zpf*eps_drive/2*1j*(qtp.tensor(id_t1, coup_op_t2).dag() - qtp.tensor(id_t1, coup_op_t2))

#Total Hamiltonian
h_0 = h_t1 + h_t2
h = h_0 + h_coup
h_tot = h + h_drive

In [169]:
eig_en, eig_states = h.eigenstates()
eig_en_tot, eig_states_tot = h_tot.eigenstates()

In [170]:
def dressed_state(
    lev: str,
    hamil: qtp.qobj.Qobj(),
    lev_t1: int,
    lev_t2: int
):
    """
    Description
    ---------------------------------------------------------
    Function that returns the dressed state
    """
    
    #eig_en_0, eig_states_0 = h_0.eigenstates()
    eig_en, eig_states = hamil.eigenstates()
    
    lev_list = [int(index) for index in lev]
    
    if lev_list[0] >= lev_t1 or lev_list[1] >= lev_t2:
        raise ValueError("Index error: indeces out of bound")
    state_0 = qtp.tensor(qtp.basis(lev_t1, lev_list[0]), qtp.basis(lev_t2, lev_list[1]))
    #en_0 = eig_en_0[state_index]
    eig_states_list = list(eig_states)
    state = eig_states_list[0]
    en = eig_en[0]
    fid = qtp.fidelity(state*state.dag(), state_0*state_0.dag())
    for state_tmp, en_tmp in zip(eig_states_list, eig_en):
        fid_tmp = qtp.fidelity(state_tmp*state_tmp.dag(), state_0*state_0.dag())
        if fid_tmp > fid:
            state = state_tmp
            fid = fid_tmp
            en = en_tmp
    loc = np.argmax(np.abs(np.array(state)) )
    phase = cmath.phase(state[loc])
    state = np.exp(-1j*phase)*state
    return en, state

To get the effective Hamiltonian we employ numerically an exact Schrieffer-Wolff. All we need to do is define the projectors
involved in the SW transformation

In [171]:
def zz_drive(
    hamil,
    lev_t1: int,
    lev_t2: int
) -> float:
    en_00 = dressed_state('00', hamil, lev_t1, lev_t2)[0]
    en_01 = dressed_state('01', hamil, lev_t1, lev_t2)[0]
    en_10 = dressed_state('10', hamil, lev_t1, lev_t2)[0]
    en_11 = dressed_state('11', hamil, lev_t1, lev_t2)[0]
    xi_zz = en_00 + en_11 - en_01 - en_10
    return xi_zz

def project_dressed(
    op: qtp.qobj.Qobj(),
    hamil: qtp.qobj.Qobj(),
    lev_t1: int,
    lev_t2: int
):
    op_proj = np.zeros([4, 4], dtype=complex)
    states = {}
    states["00"] = dressed_state('00', hamil, lev_t1, lev_t2)[1]
    states["01"] = dressed_state('01', hamil, lev_t1, lev_t2)[1]
    states["10"] = dressed_state('10', hamil, lev_t1, lev_t2)[1]
    states["11"] = dressed_state('11', hamil, lev_t1, lev_t2)[1]
    
    for key_row, row in zip(states.keys(), range(0, 4)):
        for key_col, col in zip(states.keys(), range(0, 4)):
            op_proj[row, col] = op.matrix_element(states[key_row], states[key_col])
    
    return op_proj

def projector(
    hamil: qtp.qobj.Qobj(),
    lev_t1: int,
    lev_t2: int,
    labels=["00", "01", "10", "11"]
):
    states = {}
    proj = 0
    for label in labels:
        states[label] = dressed_state(label, hamil, lev_t1, lev_t2)[1]
        proj += states[label]*states[label].dag()
    return proj, states

In [172]:
proj_dressed, states_dressed = projector(h, lev_t, lev_t)
proj_dressed_drive, states_dressed_drive = projector(h_tot, lev_t, lev_t)
ref_dressed = qtp.tensor(id_t1, id_t2) - 2*proj_dressed
ref_dressed_drive = qtp.tensor(id_t1, id_t2) - 2*proj_dressed_drive
u_sw = (ref_dressed*ref_dressed_drive).sqrtm()

In [173]:
def h_sw_exact(
    proj_0: qtp.qobj.Qobj(),
    proj: qtp.qobj.Qobj(),
    hamil: qtp.qobj.Qobj(),
    states_0: dict,
    lev_t1: int,
    lev_t2: int
    
):
    refl_0 = qtp.tensor(qtp.qeye(lev_t1), qtp.qeye(lev_t2)) - 2*proj_0
    refl = qtp.tensor(qtp.qeye(lev_t1), qtp.qeye(lev_t2)) - 2*proj
    u_sw = (refl_0*refl).sqrtm()
    h_sw = u_sw*hamil*u_sw.dag()
    h_eff = np.zeros([4, 4], dtype=complex)
    for key_row, row in zip(states_0.keys(), range(0, 4)):
        for key_col, col in zip(states_0.keys(), range(0, 4)):
            h_eff[row, col] = hamil.matrix_element(states_0[key_row], states_0[key_col])
    return h_eff

In [174]:
h_sw_eff = h_sw_exact(proj_dressed, proj_dressed_drive, h_tot, states_dressed, lev_t, lev_t)

The easiest thing is to obtain the projector on the dressed basis without the drive

In [175]:
def get_pauli_coeff(
    op: np.ndarray,
    label: str
):
    pauli = {}
    pauli["I"] = np.identity(2, dtype=complex)
    pauli["X"] = np.array([[0.0, 1.0], [1.0, 0.0]], dtype=complex)
    pauli["Y"] = np.array([[0.0, -1j*1.0], [1j*1.0, 0.0]], dtype=complex)
    pauli["Z"] = np.array([[1.0, 0.0], [0.0, -1.0]], dtype=complex)
    label_list = [x for x in label]
    pauli_op = np.kron(pauli[label_list[0]], pauli[label_list[1]])
    coeff = 1/4*np.trace(op.conj().T.dot(pauli_op))
    return coeff

In [176]:
get_pauli_coeff(h_sw_eff, "YZ")*1e3*2

(0.5595015847468636+0j)

In [177]:
def mu_yz_flx(comp_states, op) -> float:
    yz0 = get_mat_elem(op, comp_states['00'], comp_states['10'])
    yz1 = get_mat_elem(op, comp_states['01'], comp_states['11'] )
    return (np.imag(yz0 - yz1))/2

def mu_yi_flx(comp_states, op) -> float:
    yz0 = get_mat_elem(op, comp_states['00'], comp_states['10'] )
    yz1 = get_mat_elem(op, comp_states['01'], comp_states['11'] )
    return (np.imag(yz0 + yz1))/2

In [178]:
mu_yz_flx(comp_states, coupled_sys['T2'].charge_op())*eps_drive*1e3

0.5304084779769744

There seems to be a mysterious factor of 2